<a href="https://colab.research.google.com/github/yasiral/yasiral.github.io/blob/master/FLUX_Dev_DreamBooth_FT_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Information

Display the current environment's hardware specifications,

In [ ]:
!nvidia-smi
!cat /proc/cpuinfo
!cat /proc/meminfo


# DreamBooth Training Example for FLUX.1 [dev]

DreamBooth is a method to personalize text2image models like stable diffusion given just a few (3~5) images of a subject. This notebook guides you through the training procedure specifically adapted for FLUX.1 [dev], including a LoRA implementation.


## Important Setup Instructions

Before you begin, ensure that you have completed the necessary setup steps on the Hugging Face platform to access FLUX.1 [dev], which is a gated model.

1. Visit the FLUX.1 [dev] Hugging Face page.
2. Fill in the form and accept the terms to access the model.
3. Use the command below to login to Hugging Face from your local machine:


In [ ]:
!huggingface-cli login


## Installation of Dependencies

Ensure you have all the required libraries by installing them as follows:


In [ ]:
# Clone the repository and install the package and its dependencies
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .

# Navigate to the DreamBooth example directory and install example-specific requirements
%cd examples/dreambooth
!pip install -r requirements_flux.txt


### Initialize 🤗 Accelerate

To handle different environments and simplify running the training scripts, initialize an Accelerate configuration:


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()


## Dog Toy Example

First, we will download the dataset of dog images used for the training example.


In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download("diffusers/dog-example", local_dir=local_dir, repo_type="dataset")


## Training the Model

Now that the dataset is ready, let's setup the environment variables and start the training process.


In [ ]:
import os

# Set environment variables
os.environ['MODEL_NAME'] = "black-forest-labs/FLUX.1-dev"
os.environ['INSTANCE_DIR'] = "dog"
os.environ['OUTPUT_DIR'] = "trained-flux"

# Run the training script
!accelerate launch train_dreambooth_flux.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="bf16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=25 \
  --seed="0" \
  --push_to_hub


## Training with LoRA

For a more parameter-efficient training using LoRA, follow the steps below:


In [ ]:
os.environ['OUTPUT_DIR'] = "trained-flux-lora"

# Run the LoRA training script
!accelerate launch train_dreambooth_lora_flux.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="bf16" \
  --train_text_encoder \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-5 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --seed="0" \
  --push_to_hub


## Note on Memory Consumption and Performance

Remember, training such models can be resource-intensive. Consider the system requirements and setup optimizations like using 8-bit Adam or adjusting the `torch` compile mode for performance gains.
